## Exercise 01 : Basic operations

In [1]:
import pandas as pd

input_file_path = "../data/feed-views.log"

- create a dataframe views with two columns: datetime and user by reading feed-views.log
    - convert the datetime to the datetime64[ns] Dtype
    - extract the year, month, day, hour, minute, and second from the values of that column to the new columns

In [2]:
df = pd.read_csv(
    input_file_path, header=None, sep="\t", index_col=0, names=["datetime", "user"]
)
df.index = pd.to_datetime(
    df.index
)  # convert datetime col (index column) to datetime (datetime64[ns])
df["year"] = df.index.year
df["month"] = df.index.month
df["day"] = df.index.day
df["hour"] = df.index.hour
df["minute"] = df.index.minute
df["second"] = df.index.second

display(df)

,user,year,month,day,hour,minute,second
datetime,,,,,,,
2020-04-17 12:01:08.463179,artem,2020,4,17,12,1,8
2020-04-17 12:01:23.743946,artem,2020,4,17,12,1,23
2020-04-17 12:27:30.646665,artem,2020,4,17,12,27,30
2020-04-17 12:35:44.884757,artem,2020,4,17,12,35,44
2020-04-17 12:35:52.735016,artem,2020,4,17,12,35,52
...,...,...,...,...,...,...,...
2020-05-21 18:45:20.441142,valentina,2020,5,21,18,45,20
2020-05-21 23:03:06.457819,maxim,2020,5,21,23,3,6
2020-05-21 23:23:49.995349,pavel,2020,5,21,23,23,49


- create the new column daytime
    - you need to assign the particular time of day value if an hour is within a particular interval, for example, afternoon if the hour is larger than 11 and less or equal to 17
    - 0 – 3.59 night, 4 – 6.59 early morning, 7 – 10.59 morning, 11 – 16.59 afternoon, 17 – 19.59 early evening, 20 – 23.59 evening
    - use the method cut to solve this subtask
    - assign the column user as the index

In [3]:
labels = ["night", "early morning", "morning", "afternoon", "early evening", "evening"]
bins = [0, 4, 7, 11, 17, 20, 24]
df["daytime"] = pd.cut(df["hour"], bins=bins, labels=labels, right=False)
df.set_index("user", inplace=True)
display(df)

,year,month,day,hour,minute,second,daytime
user,,,,,,,
artem,2020,4,17,12,1,8,afternoon
artem,2020,4,17,12,1,23,afternoon
artem,2020,4,17,12,27,30,afternoon
artem,2020,4,17,12,35,44,afternoon
artem,2020,4,17,12,35,52,afternoon
...,...,...,...,...,...,...,...
valentina,2020,5,21,18,45,20,early evening
maxim,2020,5,21,23,3,6,evening
pavel,2020,5,21,23,23,49,evening


- calculate the number of elements in your dataframe
    - use the method count()
    - calculate the number of elements in each time of day category using the method value_counts()

In [4]:
df.count()

year       1076
month      1076
day        1076
hour       1076
minute     1076
second     1076
daytime    1076
dtype: int64

In [5]:
df.value_counts("daytime")

daytime
evening          509
afternoon        252
early evening    145
night            129
morning           36
early morning      5
Name: count, dtype: int64

- sort values in your dataframe by hour, minute, and second in ascending order (simultaneously and not one by one)

In [6]:
df.sort_values(by=["hour", "minute", "second"], ascending=True, inplace=True)
display(df)

,year,month,day,hour,minute,second,daytime
user,,,,,,,
valentina,2020,5,15,0,0,13,night
valentina,2020,5,15,0,1,5,night
pavel,2020,5,12,0,1,27,night
pavel,2020,5,12,0,1,38,night
pavel,2020,5,12,0,1,55,night
...,...,...,...,...,...,...,...
artem,2020,5,21,23,49,22,evening
anatoliy,2020,5,9,23,53,55,evening
pavel,2020,5,9,23,54,54,evening


- calculate the minimum and maximum for the hours and the mode for the daytime categories
    - calculate the maximum of hour for the rows where the time of day is night
    - calculate the minimum of hour for the rows where the time of day is morning
    - In addition to this, find out who visited the page at those hours (make one example from that)
    - calculate the mode for the hour and daytime

In [7]:
max_hour_night = df.loc[df["daytime"] == "night", "hour"].max()
min_hour_morning = df.loc[df["daytime"] == "morning", "hour"].min()
hour_mode = (
    df["hour"].mode().iloc[0]
)  # .iloc[0] gets the first element (index) in Series
daytime_mode = df["daytime"].mode().iloc[0]
user_max_night = df.loc[df["daytime"] == "night", "hour"].idxmax()
user_min_morning = df.loc[df["daytime"] == "morning", "hour"].idxmin()
print(
    f"The maximum of hour for the rows where the time of day is night : {max_hour_night}\n"
    f"The minimum of hour for the rows where the time of day is morning : {min_hour_morning}\n"
    f"The user who visited site at max night hour : {user_max_night}\n"
    f"The user who visited site at min morning hour : {user_min_morning}\n"
    f"Mode for the hour : {hour_mode}\n"
    f"Mode for the daytime : {daytime_mode}\n"
)

The maximum of hour for the rows where the time of day is night : 3
The minimum of hour for the rows where the time of day is morning : 8
The user who visited site at max night hour : konstantin
The user who visited site at min morning hour : alexander
Mode for the hour : 22
Mode for the daytime : evening



- show the 3 earliest hours in the morning and the corresponding usernames and the 3 latest hours and the usernames using nsmallest() and nlargest()

In [8]:
df.loc[df["daytime"] == "morning"].nsmallest(3, "hour")

,year,month,day,hour,minute,second,daytime
user,,,,,,,
alexander,2020,5,15,8,16,3,morning
alexander,2020,5,15,8,35,1,morning
alexander,2020,5,15,9,2,24,morning


In [9]:
df.loc[df["daytime"] == "morning"].nlargest(3, "hour")

,year,month,day,hour,minute,second,daytime
user,,,,,,,
konstantin,2020,5,16,10,2,8,morning
maxim,2020,5,14,10,8,4,morning
maxim,2020,5,14,10,8,5,morning


- use the method describe() to get the basic statistics for the columns
    - to find out what the most popular interval for visiting the page is, calculate the interquartile range for the hour by extracting values from the result of the describe() method and store it in the variable iqr

In [10]:
desc = df.describe()
display(desc)

,year,month,day,hour,minute,second
count,1076.0,1076.000000,1076.000000,1076.000000,1076.000000,1076.000000
mean,2020.0,4.870818,13.552974,16.249071,29.629182,29.500929
std,0.0,0.335557,4.906567,6.955490,17.689388,17.405506
min,2020.0,4.000000,1.000000,0.000000,0.000000,0.000000
25%,2020.0,5.000000,11.000000,13.000000,14.000000,14.000000
50%,2020.0,5.000000,13.000000,19.000000,29.000000,30.000000
75%,2020.0,5.000000,15.000000,22.000000,46.000000,45.000000
max,2020.0,5.000000,30.000000,23.000000,59.000000,59.000000


In [11]:
hour_25_percentile = desc.loc["25%", "hour"]
hour_mean = desc.loc["50%", "hour"]
hour_75_percentile = desc.loc["75%", "hour"]
iqr = hour_75_percentile - hour_25_percentile
print(f"IQR: {iqr}")

IQR: 9.0
